In [23]:
import pandas as pd
import geopandas as gpd
import pull_noaa
import read_file
import load_weather
import load_interventions

In [24]:
noaa = pull_noaa.main()
noaa = noaa[['date', 'PRCP', 'TMAX', 'TMIN', 'lat', 'lon']]

reading...
fixing date...


In [28]:
shape = gpd.read_file('../data_raw/tl_2017_us_county/tl_2017_us_county.shp')
shape = shape[['GEOID', 'NAMELSAD', 'geometry']]

In [29]:
shape.NAMELSAD = shape.NAMELSAD.str.upper()
noaa.lat = pd.to_numeric(noaa.lat)
noaa.lon = pd.to_numeric(noaa.lon)

In [30]:
noaa = gpd.GeoDataFrame(noaa, crs={'init': 'epsg:4269'}, geometry=gpd.points_from_xy(x=noaa.lon, y=noaa.lat))

In [31]:
noaa = noaa.fillna(noaa.mean())

In [32]:
weather = gpd.sjoin(noaa, shape, how='inner', op='intersects')

In [35]:
weather.columns = ['Date', 'Precipitation', 'Tempreture_max', 'Tempreture_min', 'Latitude', 'Longitude', 'Geometry', 'Index_right', 'FIPS', 'CountyName']
weather.head(5)

,Date,Precipitation,Tempreture_max,Tempreture_min,Latitude,Longitude,Geometry,Index_right,FIPS,CountyName
6277,2020-02-15,130.0,290.0,256.0,-14.3306,-170.7136,POINT (-170.71360 -14.33060),1791,60050,WESTERN DISTRICT
6278,2020-02-16,810.0,283.0,257.0,-14.3306,-170.7136,POINT (-170.71360 -14.33060),1791,60050,WESTERN DISTRICT
6279,2020-02-17,831.0,290.0,251.0,-14.3306,-170.7136,POINT (-170.71360 -14.33060),1791,60050,WESTERN DISTRICT
6280,2020-02-18,490.0,300.0,249.0,-14.3306,-170.7136,POINT (-170.71360 -14.33060),1791,60050,WESTERN DISTRICT
6281,2020-02-19,765.0,303.0,233.0,-14.3306,-170.7136,POINT (-170.71360 -14.33060),1791,60050,WESTERN DISTRICT


In [36]:
weather.shape

(1709877, 10)

In [40]:
agg_weather = weather.groupby(['FIPS', 'Date']).mean()
agg_weather.head(5)

Precipitation  Tempreture_max  Tempreture_min  Latitude  \
FIPS  Date                                                                  
01001 2020-02-19          241.0      138.211274         22.5347   32.6444   
      2020-02-20          193.0      138.211274         22.5347   32.6444   
      2020-02-21          262.0      138.211274         22.5347   32.6444   
      2020-02-24           25.0      138.211274         22.5347   32.6444   
      2020-02-25          213.0      138.211274         22.5347   32.6444   

                  Longitude  Index_right  
FIPS  Date                                
01001 2020-02-19   -86.4302         1328  
      2020-02-20   -86.4302         1328  
      2020-02-21   -86.4302         1328  
      2020-02-24   -86.4302         1328  
      2020-02-25   -86.4302         1328

In [38]:
agg_weather.shape

(268083, 6)